# Create a job for incomplete files

In [22]:
todo = '/home/users/fredner/todo.txt'

In [25]:
with open(todo) as f:
    files = [x for x in f.read().split('\n') if x]

In [26]:
len(files)

56817

In [27]:
n = len(files)//300

In [29]:
n

189

In [30]:
c = 0
for x in range(0,len(files),n):
    filename = '/scratch/groups/malgeehe/celebs/filelists_incomplete/files_{}.txt'.format(c)
    with open(filename,'w') as f:
        for file in files[x:x+n]:
            f.write(file)
            f.write('\n')
        f.close()
        c+=1

Were any of these files huge or something?

In [31]:
sizes = [os.path.getsize(x) for x in files]

In [33]:
sizes = sorted(sizes)

In [35]:
cmd = '''#!/bin/bash
#SBATCH --job-name=bigs_corenlp
#SBATCH --time=05:00:00
#SBATCH --cpus-per-task=1
#SBATCH --array=0-300
#SBATCH --mem-per-cpu=6G
ml reset
ml load java
srun java -cp "/scratch/groups/malgeehe/celebs/corenlp/corenlp_${SLURM_ARRAY_TASK_ID}/*" -Xmx5g edu.stanford.nlp.pipeline.StanfordCoreNLP -filelist /scratch/groups/malgeehe/celebs/filelists_group2/files_${SLURM_ARRAY_TASK_ID}.txt -outputDirectory /scratch/groups/malgeehe/celebs/chicago_corenlp_results_group2'''

In [36]:
with open('/scratch/groups/malgeehe/celebs/group2.sbatch','w') as f:
    f.write(cmd)
    f.close()

Figure out which files were not parsed in the initial run.

In [1]:
parsed = '/scratch/groups/malgeehe/celebs/chicago_corenlp_results'

In [2]:
import os
files = [os.path.join(parsed,x) for x in os.listdir(parsed) if x.endswith('.xml')]

In [3]:
len(files)

2909181

In [4]:
attempted = '/scratch/groups/malgeehe/celebs/filelists'

In [5]:
l = [os.path.join(attempted,x) for x in os.listdir(attempted)]

In [6]:
# get the filenames out of the filelists
def get_filenames(file):
    with open(file) as f:
        return [x.strip() for x in f.read().split('\n')]

In [7]:
test = get_filenames(l[0])

In [8]:
from multiprocessing import Pool

with Pool() as p:
    r = p.map(get_filenames,l)

In [9]:
res = [item for sublist in r for item in sublist]

In [10]:
(len(res) - len(files)) / len(res)

0.019255644828791704

About 2% of the files incomplete

In [11]:
files[0]

'/scratch/groups/malgeehe/celebs/chicago_corenlp_results/CD_20151209220449_00004_493218725.txt.xml'

In [12]:
res[0]

'/scratch/groups/malgeehe/celebs/chicago_news_batched/212/CT_20170330222852_00012_174622305.txt'

In [20]:
with open('/home/users/fredner/attempted.txt','w') as f:
    for x in res:
        f.write(x)
        f.write('\n')
    f.close()

In [21]:
with open('/home/users/fredner/completed.txt','w') as f:
    for x in files:
        f.write(x)
        f.write('\n')
    f.close()

In [13]:
att_set = [os.path.split(x)[-1] for x in res]

In [14]:
att_set[0]

'CT_20170330222852_00012_174622305.txt'

In [15]:
done_set = [os.path.split(x)[-1].split('.xml')[0] for x in files]

In [16]:
done_set[0]

'CD_20151209220449_00004_493218725.txt'

In [17]:
len(done_set)

2909181

In [18]:
att_set = set(att_set)

In [ ]:
done_set = set(done_set)

In [ ]:
todo = att_set - done_set

In [33]:
todo = [x for x in list(todo) if x]

In [34]:
len(todo)

56837

In [36]:
todo[0]

'CT_20170929192915_00004_181348191.txt'

'/scratch/groups/malgeehe/celebs/chicago_news_batched/199/CT_20170929192915_00004_181348191.txt'

In [39]:
import time

In [40]:
start = time.time()
retry = []
for x in todo[:100]:
    retry.append(next((s for s in res if x in s), None))
print(time.time()-start)

19.053706884384155


In [41]:
from multiprocessing import Pool

In [42]:
def find_path(file):
    return next((s for s in res if file in s), None)

In [ ]:
start = time.time()
with Pool() as p:
    test = p.map(find_path,todo[:100])
print(time.time()-start)

In [ ]:
len(test)

# How many files are in the data directory?

In [1]:
import os

In [4]:
len(os.listdir('/scratch/groups/malgeehe/celebs/chicago_corenlp_results/'))

3295238